In [1]:
import pandas as pd  
import numpy as np
from tqdm import tqdm  
from collections import defaultdict  
import os, math, warnings, math, pickle
from tqdm import tqdm
import faiss
import collections
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
# from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
# from tensorflow.python.keras import backend as K
# from tensorflow.python.keras.models import Model
# from tensorflow.python.keras.preprocessing.sequence import pad_sequences

# from deepmatch.models import *
# from deepmatch.utils import sampledsoftmaxloss
warnings.filterwarnings('ignore')

In [2]:
# train
train_click = pd.read_csv("data/train_click_log.csv")
articles = pd.read_csv("data/articles.csv")
articles = articles.rename(columns={'article_id': 'click_article_id'})  #重命名，方便后续match
articles_emb = pd.read_csv("data/articles_emb.csv")

# test
test_click = pd.read_csv("data/testA_click_log.csv")

train_click = train_click.append(test_click)
train_click = train_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))

train_click = train_click.sort_values('user_id')
train_click.reset_index(drop=True, inplace=True)
train_click.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,0,157507,1508211702520,4,1,17,1,25,2
1,0,30760,1508211672520,4,1,17,1,25,2
2,1,63746,1508211346889,4,1,17,1,25,6
3,1,289197,1508211316889,4,1,17,1,25,6
4,2,168401,1508211468695,4,3,20,1,25,2


In [8]:
usercf_u2u2i_recall_dict = pickle.load(open('./hybird_model/usercf_u2u2i_recall.pkl', 'rb'))

In [14]:
itemcf_u2i2i_recall_dict = pickle.load(open('./hybird_model/itemcf_u2i2i_recall.pkl', 'rb'))

In [16]:
cold_start_u2i_recall_dict = pickle.load(open('./hybird_model/cold_start_u2i_recall.pkl', 'rb'))

In [17]:
weight_dict = {'itemcf_sim_itemcf_recall': 1.0,
               'usercf_sim_itemcf_recall': 0.5,
               'cold_start_recall': 0.5}

# 定义一个多路召回的字典，将各路召回的结果都保存在这个字典当中
user_multi_recall_dict =  {'itemcf_sim_itemcf_recall': {},
                           'usercf_sim_itemcf_recall': {},
                           'cold_start_recall': {}}

In [19]:
user_multi_recall_dict['itemcf_sim_itemcf_recall'] = itemcf_u2i2i_recall_dict
user_multi_recall_dict['usercf_sim_itemcf_recall'] = usercf_u2u2i_recall_dict
user_multi_recall_dict['cold_start_recall'] = cold_start_u2i_recall_dict

In [23]:
def combine_recall_results(user_multi_recall_dict, weight_dict=None, topk=5):
    final_recall_items_dict = {}
    
    # 对每一种召回结果按照用户进行归一化，方便后面多种召回结果，相同用户的物品之间权重相加
    def norm_user_recall_items_sim(sorted_item_list):
        # 如果冷启动中没有文章或者只有一篇文章，直接返回，出现这种情况的原因可能是冷启动召回的文章数量太少了，
        # 基于规则筛选之后就没有文章了, 这里还可以做一些其他的策略性的筛选
        if len(sorted_item_list) < 2:
            return sorted_item_list
        
        min_sim = sorted_item_list[-1][1]
        max_sim = sorted_item_list[0][1]
        
        norm_sorted_item_list = []
        for item, score in sorted_item_list:
            if max_sim > 0:
                norm_score = 1.0 * (score - min_sim) / (max_sim - min_sim) if max_sim > min_sim else 1.0
            else:
                norm_score = 0.0
            norm_sorted_item_list.append((item, norm_score))
            
        return norm_sorted_item_list
    
    print('多路召回合并...')
    for method, user_recall_items in tqdm(user_multi_recall_dict.items()):
        print(method + '...')
        # 在计算最终召回结果的时候，也可以为每一种召回结果设置一个权重
        if weight_dict == None:
            recall_method_weight = 1
        else:
            recall_method_weight = weight_dict[method]
        
        for user_id, sorted_item_list in user_recall_items.items(): # 进行归一化
            user_recall_items[user_id] = norm_user_recall_items_sim(sorted_item_list)
        
        for user_id, sorted_item_list in user_recall_items.items():
            # print('user_id')
            final_recall_items_dict.setdefault(user_id, {})
            for item, score in sorted_item_list:
                final_recall_items_dict[user_id].setdefault(item, 0)
                final_recall_items_dict[user_id][item] += recall_method_weight * score  
    
    final_recall_items_dict_rank = {}
    # 多路召回时也可以控制最终的召回数量
    for user, recall_item_dict in final_recall_items_dict.items():
        final_recall_items_dict_rank[user] = sorted(recall_item_dict.items(), key=lambda x: x[1], reverse=True)[:topk]

    # 将多路召回后的最终结果字典保存到本地
    # pickle.dump(final_recall_items_dict_rank, open(os.path.join(save_path, 'final_recall_items_dict.pkl'),'wb'))

    return final_recall_items_dict_rank

In [24]:
final_recall_items_dict_rank = combine_recall_results(user_multi_recall_dict, weight_dict, topk=5)

  0%|          | 0/3 [00:00<?, ?it/s]

多路召回合并...
itemcf_sim_itemcf_recall...


 33%|███▎      | 1/3 [00:01<00:03,  1.80s/it]

usercf_sim_itemcf_recall...


 67%|██████▋   | 2/3 [00:02<00:01,  1.42s/it]

cold_start_recall...


100%|██████████| 3/3 [00:04<00:00,  1.36s/it]


In [35]:
final_recall_items_dict_rank[200000]

[(194935, 1.5),
 (237870, 0.8616326270692185),
 (336221, 0.5),
 (272143, 0.44713089585246624),
 (195645, 0.4360433901218392)]

In [33]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(final_recall_items_dict_rank.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])  # final_recall_items_dict_rank

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])
recall_df.head()

100%|██████████| 50002/50002 [00:01<00:00, 34878.12it/s]


,user_id,click_article_id,pred_score
0,249999,234698,1.500000
1,249999,336223,0.976786
2,249999,95716,0.894521
3,249999,162655,0.500000
4,249999,300082,0.368260


In [28]:
# 生成提交文件
save_path = "./"

def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)

In [34]:
# 获取测试集
test_click = pd.read_csv("data/testA_click_log.csv")
test_click = test_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
test_recall = recall_df[recall_df['user_id'].isin(test_click)]

# 生成提交文件
submit(recall_df, topk=5, model_name='final_recall')